In [2]:
import os
import mlflow.pyfunc
from flask import Flask, request, jsonify
from prometheus_client import start_http_server, Counter, Summary, Histogram

# Configuration et initialisation 
# Le nom du modèle enregistré dans le MLflow Model Registry (e.g., 'iris_random_forest')
MODEL_NAME = os.environ.get("MODEL_NAME", "iris_classifier") 
# L'étape de vie à charger (e.g., 'Production')
MODEL_STAGE = os.environ.get("MODEL_STAGE", "Production") 
# Le chemin vers le serveur de tracking (mlruns/) si local, ou l'URL distante
MLFLOW_TRACKING_URI = os.environ.get("MLFLOW_TRACKING_URI", "file:///app/mlruns") 

# --- MLflow Model Loading ---
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# L'URL MLflow pour charger la version "Production" du modèle
MODEL_URI = f"models:/{MODEL_NAME}/{MODEL_STAGE}" 

try:
    model = mlflow.pyfunc.load_model(MODEL_URI)
except Exception as e:
    print(f"Erreur de chargement du modèle: {e}")
    model = None # Fallback ou gestion d'erreur

# --- Monitoring Prometheus ---
REQUEST_COUNT = Counter(
    'http_requests_total', 'Total HTTP Requests', ['method', 'endpoint']
)
REQUEST_LATENCY = Histogram(
    'http_request_latency_seconds', 'HTTP Request Latency', ['method', 'endpoint']
)

# --- Flask App ---
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
@REQUEST_LATENCY.time()
def predict():
    REQUEST_COUNT.labels(method='post', endpoint='/predict').inc()
    
    if model is None:
        return jsonify({"error": "Model not loaded"}), 500

    try:
        data = request.get_json(force=True)
        
        # Supposons que 'data' est une liste de features, 
        # le modèle prend en charge la prédiction
        prediction = model.predict(data.get('features'))

        return jsonify({"prediction": prediction.tolist()})

    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Point de terminaison pour les métriques, utilisé par Prometheus
@app.route('/metrics')
def metrics():
    from prometheus_client import generate_latest, CONTENT_TYPE_LATEST
    return generate_latest(), 200, {'Content-Type': CONTENT_TYPE_LATEST}


if __name__ == '__main__':
    # Démarrer le serveur de métriques Prometheus sur un port séparé (e.g., 8000)
    start_http_server(8000) 
    print("Prometheus metrics server started on port 8000")
    
    # Lancer l'API Flask
    app.run(host='0.0.0.0', port=8080)

c:\Users\Imran\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
c:\Users\Imran\AppData\Local\Programs\Python\Python310\lib\site-packages\mlflow\tracking\_model_registry\utils.py:215: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)


Erreur de chargement du modèle: Registered Model with name=iris_classifier not found
Prometheus metrics server started on port 8000
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.72:8080
Press CTRL+C to quit
